In [2]:
from __future__ import division, print_function

import twitter
import json
from operator import itemgetter
import string
from time import sleep
from gensim import corpora, models
import nltk
from nltk import word_tokenize, FreqDist

In [11]:
CONSUMER_KEY = 'mn4YRu8l9vnoGtIwDn8wDBhJb'
CONSUMER_SECRET = 'vB2CgIWUy6f27Gw806d6HQ4WhQMxKgz06kLybZPcwH6sL7PnJ9'
OAUTH_TOKEN = '4804645274-F9D6CIuGlOY4uIthQh1IijuRLLKGEt2aJ84Q8Al'
OAUTH_TOKEN_SECRET = 'RPqhg5a14PzhCe1NdrpI3S4XOX2kGVtT0fm7LJSc514jz'

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)
api = twitter.Twitter(auth=auth)

def retrieve_tag(key_word):

    print("Beginning retrieval of " + key_word)
    try:
        timeline = api.search.tweets(q=key_word, lang='en', count=100)
    except:
        print("Reached rate limit; sleeping 15 minutes")
        sleep(900)
        timeline = api.search.tweets(q=key_word, lang='en', count=100)

    ntweets = len(timeline['statuses'])
    if ntweets == 0:
        return timeline
    #### change 30000 to the total number of tweets you want
    while ntweets > 0 and len(timeline['statuses']) < 30000:
        min_id = min([tweet["id"] for tweet in timeline['statuses']])
        try:
            next_timeline = api.search.tweets(q=key_word, lang='en', count=100, max_id=min_id - 1)
        except:
            print("Reached rate limit; sleeping 15 minutes")
            sleep(900)
            print("Restarting")
            next_timeline = api.search.tweets(q=key_word, lang='en', count=100, max_id=min_id - 1)

        ntweets = len(next_timeline['statuses'])
        timeline['statuses'] += next_timeline['statuses']
    return timeline

In [12]:
uber = retrieve_tag('#uber')

Beginning retrieval of #uber
Reached rate limit; sleeping 15 minutes
Restarting


In [15]:
len(uber['statuses'])

30023

In [14]:
with open("hashtagUber.json", "w") as outfile:
    json.dump(uber, outfile, indent=4, sort_keys=True)

In [16]:
with open("hashtagUber.json") as infile:
    tweet_uber = json.load(infile)

In [13]:
len(tweet_uber['statuses'])

30023

In [14]:
tweet_uber.keys()

dict_keys(['statuses', 'search_metadata'])

In [15]:
tweet_uber['statuses'][1]['entities']

{'hashtags': [{'indices': [19, 24], 'text': 'uber'},
  {'indices': [45, 51], 'text': 'toxic'},
  {'indices': [67, 74], 'text': 'ubered'}],
 'symbols': [],
 'urls': [{'display_url': 'twitter.com/taximassive/st…',
   'expanded_url': 'https://twitter.com/taximassive/status/695011911725875200',
   'indices': [75, 98],
   'url': 'https://t.co/oB4Q7zB7dQ'}],
 'user_mentions': [{'id': 3001518083,
   'id_str': '3001518083',
   'indices': [3, 17],
   'name': 'PETE B',
   'screen_name': 'peterbyrne822'}]}

In [17]:
texts = [a['text'] for a in tweet_uber['statuses']]

In [18]:
time = [a['created_at'] for a in tweet_uber['statuses']]

In [14]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 

In [19]:
texts = ','.join(texts) # convert list to string

In [20]:
tokens = preprocess(texts)

In [2]:
with open("tokens.json", "w") as outfile:
    json.dump(tokens, outfile, indent=4, sort_keys=True)

NameError: name 'json' is not defined

In [3]:
with open("tokens.json") as infile:
    tokens = json.load(infile)

In [4]:
import operator 
import json
from collections import Counter

In [5]:
count_all = Counter()
    
# Update the counter
count_all.update(tokens)


In [6]:
# Print the first 5 most frequent words
print(count_all.most_common(5))

[(',', 20975), ('#Uber', 20335), ('.', 19541), (':', 16499), ('to', 8974)]


In [52]:
from nltk.corpus import stopwords
import string
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via','…']

In [53]:
def all_punct(x):
    return(all([char in stop for char in x]))

def my_tokenize(text):
    return([w for w in tokens if not all_punct(w)])

In [58]:
tokens = my_tokenize(texts)
tokens = nltk.Text(tokens)
hashtagtokens = [a for a in tokens if a[0] == '#']
fdist = FreqDist(hashtagtokens)
fdist.most_common(50)

[('#Uber', 20335),
 ('#uber', 7959),
 ('#taxis', 2670),
 ('#SuperBowl50', 2398),
 ('#Free', 2394),
 ('#FREERIDE', 2388),
 ('#Code', 2387),
 ('#RIDE', 2386),
 ('#Promo', 2386),
 ('#GOPATS', 2384),
 ('#SALE', 2384),
 ('#PATRIOTS', 2384),
 ('#WINTER', 2384),
 ('#SB50', 1824),
 ('#UBER', 1362),
 ('#Paris', 737),
 ('#promocode', 673),
 ('#UberTaxiWars', 666),
 ('#tfl', 587),
 ('#yegcc', 550),
 ('#startup', 543),
 ('#Lyft', 496),
 ('#France', 491),
 ('#lyft', 477),
 ('#NYC', 444),
 ("#Uber's", 443),
 ('#taxi', 432),
 ('#yeg', 398),
 ('#branding', 368),
 ('#tech', 367),
 ('#FREE', 306),
 ('#logo', 294),
 ('#free', 266),
 ('#weed', 248),
 ('#rideshare', 240),
 ('#Edmonton', 235),
 ('#ubered', 233),
 ('#design', 218),
 ('#London', 212),
 ('#Taxi', 196),
 ('#clashes', 189),
 ('#Clashes', 179),
 ('#uberlogo', 172),
 ('#uberstrike', 171),
 ('#greve', 169),
 ('#NBA', 157),
 ('#Tuscaloosa', 156),
 ('#minicab', 150),
 ('#travel', 149),
 ('#sharingeconomy', 140)]